# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [55]:
# import libraries
import re
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
nltk.download(['punkt', 'wordnet','stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [56]:
# load data from database
engine = create_engine('sqlite:///data/InsertDatabaseName.db')
df = pd.read_sql_table("disaster_messages", con=engine)
X = df['message']
Y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [57]:
def tokenize(text):
    """
    Tokenizes input text by converting it to lowercase, removing stopwords,
    and lemmatizing each token.
    
    Args:
    - text (str): the input text to be tokenized
    
    Returns:
    - tokens (list of str): the list of tokens after text processing
    """
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [58]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=40)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [60]:
y_pred = pipeline.predict(X_test)

In [63]:
print(classification_report(y_test, y_pred, target_names=Y.columns))

                        precision    recall  f1-score   support

               related       0.84      0.92      0.88      3993
               request       0.81      0.43      0.56       912
                 offer       0.00      0.00      0.00        26
           aid_related       0.74      0.58      0.65      2164
          medical_help       0.69      0.07      0.13       401
      medical_products       0.62      0.11      0.19       270
     search_and_rescue       0.64      0.05      0.09       151
              security       0.00      0.00      0.00        91
              military       0.58      0.11      0.19       168
           child_alone       0.00      0.00      0.00         0
                 water       0.84      0.33      0.47       333
                  food       0.85      0.46      0.60       559
               shelter       0.84      0.30      0.44       460
              clothing       0.80      0.16      0.27        73
                 money       0.75      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [64]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f728b64d9d8>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [67]:
parameters = {'clf__estimator__n_estimators': [10, 50, 100]}

In [68]:
cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2, n_jobs=-1, cv=2)
cv

GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [10, 50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [69]:
cv.fit(X_train, y_train)

Fitting 2 folds for each of 3 candidates, totalling 6 fits
[CV] clf__estimator__n_estimators=10 .................................
[CV] clf__estimator__n_estimators=10 .................................
[CV] clf__estimator__n_estimators=50 .................................
[CV] clf__estimator__n_estimators=50 .................................
[CV] clf__estimator__n_estimators=100 ................................
[CV] clf__estimator__n_estimators=100 ................................
[CV] .................. clf__estimator__n_estimators=10, total=23.2min
[CV] .................. clf__estimator__n_estimators=10, total=23.3min
[CV] .................. clf__estimator__n_estimators=50, total=34.4min


[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed: 42.3min remaining: 42.3min


[CV] .................. clf__estimator__n_estimators=50, total=34.5min
[CV] ................. clf__estimator__n_estimators=100, total=43.8min
[CV] ................. clf__estimator__n_estimators=100, total=44.0min


[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed: 49.6min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [10, 50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [70]:
cv.best_params_

{'clf__estimator__n_estimators': 100}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [71]:
y_pred = cv.predict(X_test)

In [72]:
print(classification_report(y_test, y_pred, target_names=Y.columns))

                        precision    recall  f1-score   support

               related       0.83      0.96      0.89      3993
               request       0.87      0.48      0.62       912
                 offer       0.00      0.00      0.00        26
           aid_related       0.77      0.69      0.73      2164
          medical_help       0.68      0.07      0.13       401
      medical_products       0.76      0.09      0.17       270
     search_and_rescue       0.69      0.06      0.11       151
              security       0.00      0.00      0.00        91
              military       0.47      0.04      0.08       168
           child_alone       0.00      0.00      0.00         0
                 water       0.90      0.34      0.49       333
                  food       0.82      0.53      0.65       559
               shelter       0.84      0.32      0.46       460
              clothing       0.76      0.18      0.29        73
                 money       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [73]:
import pickle
filename = 'models/classifier.pkl'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.